In [1]:
import json

print("Loading scene graphs ...")
#with open('sceneGraphs/train_sceneGraphs.json') as f:
with open('/../../nobackup/s144454/dataset/scene_graphs/train_sceneGraphs.json') as f:
    scene_graph = json.load(f)
print("Finished loading scene graphs!")

Loading scene graphs ...
Finished loading scene graphs!


In [2]:
from nltk.corpus import wordnet as wn

#vehicle = wn.synset('vehicle.n.01')
#car = wn.synset('car.n.01')

def is_hypernym(child, parent):
    child = lemmatizer.lemmatize(child)
    parent = lemmatizer.lemmatize(parent)
    parent_synset = wn.synsets(parent)
    if not parent_synset:
        return False
    else:
        tmp = None
        for p in parent_synset:
            if '.n.' in p.name():
                tmp = p
                break
        if not tmp:
            return False
        else:
            parent_synset = tmp
    child_synset = wn.synsets(child)
    for s in parent_synset.closure(lambda s:s.hyponyms(), depth=9):
        for i, c in enumerate(child_synset):
            if c == s:
                return True
    return False

In [3]:
try:
    import ujson as json
except ImportError:
    try:
        import simplejson as json
    except ImportError:
        import json

import time

raw_data = {}
start_time = time.time()
with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_0.json') as f:
    raw_data.update(json.load(f))
print("finished 1 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_1.json') as f:
    raw_data.update(json.load(f))
print("finished 2 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_2.json') as f:
    raw_data.update(json.load(f))
print("finished 3 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_3.json') as f:
    raw_data.update(json.load(f))
print("finished 4 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_4.json') as f:
    raw_data.update(json.load(f))
print("finished 5 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_5.json') as f:
    raw_data.update(json.load(f))
print("finished 6 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_6.json') as f:
    raw_data.update(json.load(f))
print("finished 7 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_7.json') as f:
    raw_data.update(json.load(f))
print("finished 8 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_8.json') as f:
    raw_data.update(json.load(f))
print("finished 9 requires {}".format(time.time() - start_time))

with open('/../../nobackup/s144454/dataset/questions/train_all_questions/train_all_questions_9.json') as f:
    raw_data.update(json.load(f))
print("finished 10 requires {}".format(time.time() - start_time))

finished 1 requires 67.7737295627594
finished 2 requires 151.10511350631714
finished 3 requires 274.93123984336853
finished 4 requires 363.830593585968
finished 5 requires 482.2301061153412
finished 6 requires 631.7496457099915
finished 7 requires 666.8137555122375
finished 8 requires 847.355530500412
finished 9 requires 882.0872075557709
finished 10 requires 1089.01167678833


In [3]:
import re
import nltk
import json
import sys
import json

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

def add1(string, extra):
    #nums = string[1:-1].split(',')
    #nums = [str(int(_) + extra) for _ in nums]
    new_string = ""
    for c in string:
        if c.isdigit():
            new_string += str(int(c) + extra)
        else:
            new_string += c
    return new_string

def eq(string, name, value=False):
    if name is None:
        return True
    else:
        if ' ' in string and ' ' in name:
            return string == name
        elif ' ' in string:
            return name in string.split(' ')
        elif ' ' in name:
            return string in name.split(' ')
        else:
            str1 = lemmatizer.lemmatize(string)
            str2 = lemmatizer.lemmatize(name)
            if str1 == str2:
                return True
            else:
                return hypernym(str1, str2)
        
def filter_field(string):
    #pattern = r'white|red|black|green|silver|gold|khaki|gray|blue|yellow|dark|tan|brown|orange|purple|beige|gray'
    output = re.search(r' ([^ ]+)\b', string).group()[2:]
    if 'not(' in output:
        return re.search(r'\(.+$', output).group()[1:], True
    else:
        return output, False

def filter_parenthesis(string):
    objects = re.search(r'\(.+\)', string).group()[1:-1]
    if objects == '-':
        return '[]'
    else:
        return '[{}]'.format(objects)

def filter_squre(string):
    indexes = re.search(r'\[.+\]', string).group()
    if ',' in indexes:
        return ','.join(['[{}]'.format(_.strip()) for _ in indexes[1:-1].split(',')])
    else:
        return indexes

def extract_rel(string):    
    subject = re.search(r'^([^,]+),', string).group()[:-1]    
    relation = re.search(r',(.+),', string).group()[1:-1]
    try:
        o_s = re.search(r',(o|s) ', string).group()[1:-1]
        if 's' in o_s:
            return subject, relation, True 
        else:
            return subject, relation, False
    except:
        return subject, relation, None
    
def extract_query_key(string):
    if 'name' in string:
        return 'name'
    elif 'hposition' in string:
        return 'hposition'
    elif 'vposition' in string:
        return 'vposition'
    else:
        return 'attributes'

def split_rel(string):
    subject = re.search(r'([^,]+),', string).group()[:-1]
    relation1 = re.search(r',(.+)\|', string).group()[1:-1]
    relation2 = re.search(r'\|(.+),', string).group()[1:-1]
    o_s = re.search(r',(o|s)', string).group()[1:-1]
    if 's' in o_s:
        return subject, relation1, relation2, True 
    else:
        return subject, relation1, relation2, False

def split_attr(string):
    attr1 = re.search(r'(.+)\|', string).group()[2:-1]
    attr2 = re.search(r'\|(.+) ', string).group()[1:-1]    
    return attr1, attr2

In [37]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/s144454/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def preprocess(raw_data, output_path, formal=False):
    symbolic_programs = []
    from nltk.stem import WordNetLemmatizer 
    lemmatizer = WordNetLemmatizer()
        
    keys = list(raw_data.keys())
    print("total {} programs".format(len(keys)))
    success, fail = 0, 0
    
    for idx in range(len(keys)):
        imageId = raw_data[keys[idx]]['imageId']
        question = raw_data[keys[idx]]['question']
        program = raw_data[keys[idx]]['semantic']
        answer = raw_data[keys[idx]]['answer']

        new_programs = []
        extra = 0
        try:
            for i, prog in enumerate(program):
                if prog['dependencies']:
                    subject = ",".join(["[{}]".format(_) for _ in prog['dependencies']])
                    subject = add1(subject, extra)

                if '(' in prog['argument'] and ')' in prog['argument'] and 'not(' not in prog['argument']:
                    result = filter_parenthesis(prog['argument'])
                else:
                    result = '?'

                if prog['operation'] == 'select':
                    if prog['argument'] == 'scene':
                        #new_programs.append('{}=scene()'.format(result))
                        flag = 'full'
                    else:
                        #print("TESTING BEFORE")
                        #print(lemmatizer.lemmatize(prog['argument'].split(' ')[0]))
                        new_programs.append('{}=select({})'.format(result, lemmatizer.lemmatize(prog['argument'].split(' ')[0])))
                        #print("TESTING AFTER {}".format(new_programs[-1]))
                        flag = 'partial'

                elif prog['operation'] == 'relate':
                    #print prog['argument']
                    name, relation, reverse = extract_rel(prog['argument'])
                    if reverse == None:
                        new_programs.append('{}=relate({}, {}, {})'.format(result, subject, relation, name))
                    else:
                        if reverse:
                            if name != '_':
                                name = lemmatizer.lemmatize(name)
                                new_programs.append('{}=relate_inv_name({}, {}, {})'.format(result, subject, relation, name))
                            else:
                                new_programs.append('{}=relate_inv({}, {})'.format(result, subject, relation))
                        else:
                            if name != '_':
                                name = lemmatizer.lemmatize(name)
                                new_programs.append('{}=relate_name({}, {}, {})'.format(result, subject, relation, name))
                            else:
                                new_programs.append('{}=relate({}, {})'.format(result, subject, relation))

                elif prog['operation'].startswith('query'):
                    if prog['argument'] == "hposition":
                        new_programs.append('{}=query_h({})'.format(result, subject))
                    elif prog['argument'] == "vposition":
                        new_programs.append('{}=query_v({})'.format(result, subject))

                    elif prog['argument'] == "name":
                        new_programs.append('{}=query_n({})'.format(result, subject))
                    else:
                        if flag == 'full':
                            new_programs.append('{}=query_f({})'.format(result, prog['argument']))
                        else:
                            new_programs.append('{}=query({}, {})'.format(result, subject, prog['argument']))

                elif prog['operation'] == 'exist':
                    new_programs.append('{}=exist({})'.format(result, subject))

                elif prog['operation'] == 'or':
                    new_programs.append('{}=or({})'.format(result, subject))

                elif prog['operation'] == 'and':
                    new_programs.append('{}=and({})'.format(result, subject))

                elif prog['operation'].startswith('filter'):
                    if prog['operation'] == 'filter hposition':
                        new_programs.append('{}=filter_h({}, {})'.format(result, subject, prog['argument']))

                    elif prog['operation'] == 'filter vposition':
                        new_programs.append('{}=filter_h({}, {})'.format(result, subject, prog['argument']))

                    else:
                        negative = 'not(' in prog['argument']
                        if negative:
                            new_programs.append('{}=filter_not({}, {})'.format(result, subject, prog['argument'][4:-1]))
                        else:
                            new_programs.append('{}=filter({}, {})'.format(result, subject, prog['argument']))

                elif prog['operation'].startswith('verify'):
                    if prog['operation'] == 'verify':
                        #new_programs.append('{}=verify({}, {})'.format(result, subject, prog['argument']))
                        new_programs.append('{}=filter({}, {})'.format(result, subject, prog['argument']))
                        new_programs.append('{}=exist({})'.format(result, add1(subject, 1)))
                        extra += 1
                    elif prog['operation'] == 'verify hposition':
                        #new_programs.append('{}=verify_h({}, {})'.format(result, subject, prog['argument']))
                        new_programs.append('{}=filter_h({}, {})'.format(result, subject, prog['argument']))
                        new_programs.append('{}=exist({})'.format(result, add1(subject, 1)))
                        extra += 1
                    elif prog['operation'] == 'verify vposition':
                        #new_programs.append('{}=verify_v({}, {})'.format(result, subject, prog['argument']))
                        new_programs.append('{}=filter_v({}, {})'.format(result, subject, prog['argument']))
                        new_programs.append('{}=exist({})'.format(result, add1(subject, 1)))
                        extra += 1
                    elif prog['operation'] == 'verify rel':
                        name, relation, reverse = extract_rel(prog['argument'])
                        name = lemmatizer.lemmatize(name)
                        if reverse:
                            #new_programs.append('{}=verify_rel_inv({}, {}, {})'.format(result, subject, relation, name))
                            new_programs.append('{}=relate_inv_name({}, {}, {})'.format(result, subject, relation, name))
                            new_programs.append('{}=exist({})'.format(result, add1(subject, 1)))
                            extra += 1
                        else:
                            #new_programs.append('{}=verify_rel({}, {}, {})'.format(result, subject, relation, name))
                            new_programs.append('{}=relate_name({}, {}, {})'.format(result, subject, relation, name))
                            new_programs.append('{}=exist({})'.format(result, add1(subject, 1)))
                            extra += 1
                    else:
                        if flag == 'full':
                            new_programs.append('{}=verify_f({})'.format(result, prog['argument']))
                        else:
                            #new_programs.append('{}=verify({}, {})'.format(result, subject, prog['argument']))
                            new_programs.append('{}=filter({}, {})'.format(result, subject, prog['argument']))
                            new_programs.append('{}=exist({})'.format(result, add1(subject, 1)))
                            extra += 1

                elif prog['operation'].startswith('choose'):
                    if prog['operation'] == 'choose':
                        attr1, attr2 = prog['argument'].split('|')
                        if flag == "full":
                            new_programs.append('{}=choose_f({}, {})'.format(result, attr1, attr2))
                        else:
                            new_programs.append('{}=choose({}, {}, {})'.format(result, subject, attr1, attr2))

                    elif prog['operation'] == 'choose rel':
                        name, relation1, relation2, reverse = split_rel(prog['argument'])
                        name = lemmatizer.lemmatize(name)
                        if reverse:
                            new_programs.append('{}=choose_rel({}, {}, {}, {})'.format(result, subject, name, relation1, relation2))
                        else:
                            new_programs.append('{}=choose_rel_inv({}, {}, {}, {})'.format(result, subject, name, relation1, relation2))

                    elif prog['operation']  == 'choose hposition':
                        attr1, attr2 = prog['argument'].split('|')
                        new_programs.append('{}=choose_h({}, {}, {})'.format(result, subject, attr1, attr2))

                    elif prog['operation'] == 'choose vposition':
                        attr1, attr2 = prog['argument'].split('|')
                        new_programs.append('{}=choose_v({}, {}, {})'.format(result, subject, attr1, attr2))

                    elif prog['operation'] == 'choose name':
                        name1, name2 = prog['argument'].split('|')
                        name1 = lemmatizer.lemmatize(name1)
                        name2 = lemmatizer.lemmatize(name2)
                        new_programs.append('{}=choose_n({}, {}, {})'.format(result, subject, name1, name2))

                    elif prog['operation'] == 'choose healthier':
                        new_programs.append('{}=choose({}, {})'.format(result, subject, 'healthier'))
                    elif prog['operation'] == 'choose less healthy':
                        new_programs.append('{}=choose({}, {})'.format(result, subject, 'less healthy'))
                    elif prog['operation'] == 'choose younger':
                        new_programs.append('{}=choose({}, {})'.format(result, subject, 'younger'))
                    elif prog['operation'] == 'choose older':
                        new_programs.append('{}=choose({}, {})'.format(result, subject, 'older'))
                    else:
                        attr1, attr2 = prog['argument'].split('|')
                        if flag == "full":
                            new_programs.append('{}=choose_f({}, {})'.format(result, attr1, attr2))
                        else:
                            new_programs.append('{}=choose({}, {}, {})'.format(result, subject, attr1, attr2))

                elif prog['operation'].startswith('different'):
                    if prog['operation'] == 'different color':
                        new_programs.append('{}=different_color({})'.format(result, subject))
                    else:
                        new_programs.append('{}=different({})'.format(result, subject))

                elif prog['operation'].startswith('same'):
                    if prog['operation'] == 'same color':
                        new_programs.append('{}=same_color({})'.format(result, subject))
                    else:
                        new_programs.append('{}=same({})'.format(result, subject))

                elif prog['operation'] == 'common':
                    new_programs.append('{}=common({})'.format(result, subject))

                else:
                    raise ValueError("Unseen Function {}".format(prog))
                #if answer == "yes":
                #    answer = True
                #elif answer == "no":
                #    answer = False
                #elif 'choose' in new_programs[-1]:
                #    _, _, arguments = parse_program(new_programs[-1])
                #    if answer not in arguments:
                #        import pdb
                #        pdb.set_trace()   
                #elif answer == "right" and 'choose' in new_programs[-1]:
                #    answer = 'to the right of'
                #elif answer == "left" and 'choose' in new_programs[-1]:
                #    answer = 'to the left of'

            symbolic_programs.append((imageId, question, new_programs, answer))
            success += 1

        except Exception:
            print("Operation: {}".format(prog))
            fail += 1

        if idx % 10000 == 0:
            sys.stdout.write("finished {}/{} \r".format(success, fail))
    
    print("finished {}/{}".format(success, fail))
    print("Writing programs to json ...")
    with open(output_path, 'w') as f:
        json.dump(symbolic_programs, f, indent=2)

In [6]:
#preprocess(raw_dev_data, 'questions/dev_programs.json')
#preprocess(raw_data, 'questions/train_programs.json')
import time
start_time = time.time()
print("Preprocessing data ...")


with open('/../../nobackup/s144454/dataset/questions/train_balanced_questions.json') as f:
    raw_data = json.load(f)
preprocess(raw_data, '/../../nobackup/s144454/processed/train_balanced_programs.json')

with open('/../../nobackup/s144454/dataset/questions/testdev_balanced_questions.json') as f:
    raw_dev_data = json.load(f)

preprocess(raw_dev_data, '/../../nobackup/s144454/processed/testdev_balanced_programs.json')



print("Loading validation data ...")
with open('/../../nobackup/s144454/dataset/questions/val_balanced_questions.json') as f:
    raw_dev_data = json.load(f)


preprocess(raw_dev_data, '/../../nobackup/s144454/processed/val_balanced_programs.json')



#preprocess(raw_data, '/../../nobackup/s144454/processed/train_all_programs.json')
print("finished preprocessing data - requires {}".format(time.time() - start_time))

Preprocessing data ...
total 943000 programs
Operation: {'operation': 'choose taller', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose shorter', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose shorter', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose larger', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose taller', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose shorter', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose smaller', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose taller', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose lower', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose smaller', 'dependencies': [0, 1], 'argument': ''}
Operation: {'operation': 'choose taller', 'dependencies': [0, 1], 'argument': ''}
finished 942989/11 
Writing programs to json ...


In [51]:
import json

#with open('questions/train_programs.json') as f:
#    data = json.load(f)

#with open('sceneGraphs/train_sceneGraphs.json') as f:
#    scene_graph = json.load(f)
print("Loading programs ...")
with open('/../../nobackup/s144454/processed/train_all_programs.json') as f:
    data = json.load(f)

print("Loading scene graphs ...")
with open('/../../nobackup/s144454/dataset/scene_graphs/train_sceneGraphs.json') as f:
    scene_graph = json.load(f)

Loading programs ...
Loading scene graphs ...


In [9]:
def parse_program(string):
    if '=' in string:
        result, function = string.split('=')
    else:
        function = string
        result = "?"

    func, arguments = function.split('(')
    if len(arguments) == 1:
        return result, func, []
    else:
        arguments = list(map(lambda x: x.strip(), arguments[:-1].split(',')))
        return result, func, arguments

In [88]:
import API_provider
#from importlib import reload
#reload(API_provider)

success = 0
fail = 0
for idx, entry in enumerate(data):
    imageId = entry[0]
    graph = scene_graph[imageId]
    answer = entry[3]
    execution_buffer = []
    program = entry[2]
    for sub_program in program:
        _, func, arguments = parse_program(sub_program)
        inputs = [graph]
        for arg in arguments:
            if '[' in arg and ']' in arg:
                inputs.append(execution_buffer[int(arg[1:-1])])
            else:
                inputs.append(arg)
        #try:
        #except Exception:
        execution_buffer.append(returns)
    if type(execution_buffer[-1]) == tuple:
        if type(execution_buffer[-1][0]) == tuple:
            execution_buffer[-1] = execution_buffer[-1][0]
        if True in execution_buffer[-1]:
            execution_buffer[-1] = 'yes'
        elif False in execution_buffer[-1]:
            execution_buffer[-1] = 'no'
    if execution_buffer[-1] != answer:
        fail += 1
        print("ERROR", idx, imageId, program, "@", execution_buffer, "@", answer)
        #print imageId, program, "@", execution_buffer, "@", answer
    else:
        success += 1
    
    if idx == 10000:
        break
        
print("success = {}, fail = {}".format(success, fail))

Output: None, Answer: white
ERROR 29 2350451 ['[2579149]=select(comforter)', '?=filter_h([0], bottom)', '?=query([1], color)'] @ [['2579149'], [], None] @ white
Output: no, Answer: yes
ERROR 72 4363 ['[4370111]=select(picture)', '?=filter_h([0], top)', '[4370118]=relate_inv_name([1], to the left of, vase)', '?=exist([2])'] @ [['4370111'], [], [], 'no'] @ yes
Output: to the left of, Answer: left
ERROR 74 2327198 ['[980889]=select(desk)', '[980886]=relate_inv_name([0], on, monitor)', '?=filter([1], black)', '[980888]=choose_rel_inv([2], cup, to the left of, to the right of)'] @ [['980889'], ['980886'], ['980886'], 'to the left of'] @ left
Output: None, Answer: large
ERROR 97 2404517 ['[377100]=select(window)', '[377111]=relate_name([0], on, church)', '?=query([1], size)'] @ [['377100', '377101', '377118'], [], None] @ large
Output: ([], None), Answer: no
ERROR 168 2384324 ['[1310826]=select(weed)', '[1310811]=relate_inv_name([0], in front of, animal)', '?=filter([1], dry)', '?=exist([2])

Output: to the right of, Answer: right
ERROR 1747 2333837 ['[3398663]=select(omelette)', '[3398662]=relate_name([0], on, plate)', '[2880339]=choose_rel_inv([1], cell phone, to the right of, to the left of)'] @ [['3398663'], ['3398662'], 'to the right of'] @ right
Output: ([], None), Answer: no
ERROR 1810 2342759 ['[931082]=select(cabinet)', '[931079]=relate_inv_name([0], to the left of, stove)', '?=filter([1], black)', '?=exist([2])', '?=filter([2], wood )', '?=exist([3])', '?=and([4],[5])'] @ [['931100', '931082'], ['931079'], ['931079'], (True, '931079'), [], (True, True), ([], None)] @ no
Output: ([], None), Answer: no
ERROR 1811 2342759 ['[931079]=select(appliance)', '?=filter([0], large)', '?=filter([1], black)', '?=exist([2])', '?=filter([2], wood )', '?=exist([3])', '?=and([4],[5])'] @ [['931079'], ['931079'], ['931079'], (True, '931079'), [], (True, True), ([], None)] @ no
Output: ([], None), Answer: no
ERROR 1870 2357707 ['[809501]=select(bed)', '[809499]=relate_inv_name([0], 

ERROR 4077 2352171 ['[855620]=select(shirt)', '[855619]=relate_inv_name([0], wearing, person)', '[855623]=choose_rel_inv([1], shelf, to the left of, to the right of)'] @ [['855620', '855633', '855629'], ['855619'], 'to the left of'] @ left
Output: bus, Answer: cars
ERROR 4079 2365540 ['[756627]=select(bus)', '[756628]=relate_name([0], above, road)', '[756630]=relate_inv_name([1], on, vehicle)', '?=query_n([2])'] @ [['756627'], ['756628'], ['756627', '756630'], 'bus'] @ cars
Output: yes, Answer: no
ERROR 4085 2402555 ['[388566]=select(street)', '?=filter_not([0], white)', '?=exist([1])', '[]=select(fire)', '?=filter_not([3], white)', '?=exist([4])', '?=or([2],[5])'] @ [['1134504', '388566'], ['1134504'], (True, '1134504'), [], [], (False, None), 'yes'] @ no
Output: ([], None), Answer: no
ERROR 4096 2319661 ['[999933]=select(table)', '[999937]=relate_inv_name([0], on, plate)', '?=filter([1], orange)', '?=exist([2])', '?=filter([2], porcelain )', '?=exist([3])', '?=and([4],[5])'] @ [['999

ERROR 6034 2385356 ['[688519]=select(plate)', '[688517]=relate_inv_name([0], on top of, dessert)', '?=filter([1], chocolate)', '?=filter([2], green)', '?=exist([3])', '?=filter([3], triangular )', '?=exist([4])', '?=and([5],[6])'] @ [['688519'], ['688517'], ['688517'], [], (False, None), [], (True, False), ([], None)] @ no
Output: ([], None), Answer: no
ERROR 6035 2385356 ['[688517]=select(cake)', '?=filter([0], green)', '?=exist([1])', '?=filter([1], triangular )', '?=exist([2])', '?=and([3],[4])'] @ [['688517'], [], (False, None), [], (True, False), ([], None)] @ no
Output: ([], None), Answer: no
ERROR 6046 2410667 ['[212194]=select(sky)', '?=filter([0], clear)', '?=filter([1], black)', '?=exist([2])', '?=filter([2], bright )', '?=exist([3])', '?=and([4],[5])'] @ [['212194'], ['212194'], [], (False, None), [], (True, False), ([], None)] @ no
Output: ([], None), Answer: no
ERROR 6049 2410667 ['[212194]=select(sky)', '?=filter([0], bright)', '?=exist([1])', '?=filter([1], partly cloudy

ERROR 8040 2334996 ['[2089579]=select(girl)', '?=filter_h([0], left)', '[2270327]=choose_rel_inv([1], cow, to the right of, to the left of)'] @ [['2089579'], ['2089579'], 'to the right of'] @ right
Output: to the right of, Answer: right
ERROR 8041 2334996 ['[2089579]=select(person)', '?=filter_h([0], left)', '[2270327]=choose_rel_inv([1], cow, to the right of, to the left of)'] @ [['2089579', '2270327'], ['2089579', '2270327'], 'to the right of'] @ right
Output: yes, Answer: no
ERROR 8068 2417784 ['[2726643]=select(racket)', '?=filter_h([0], left)', '[3560089]=relate_inv_name([1], to the left of, player)', '[3560089]=exist([2])'] @ [['3290452', '2726643'], ['3290452', '2726643'], ['3560089'], 'yes'] @ no
Output: to the left of, Answer: right
ERROR 8069 2417784 ['[2726643]=select(racket)', '?=filter_h([0], left)', '[3560089]=choose_rel_inv([1], player, to the right of, to the left of)'] @ [['3290452', '2726643'], ['3290452', '2726643'], 'to the left of'] @ right
Output: to the left of, 

In [7]:
hypernyms = {}
count = 0
#for (w, freq) in question_word_counter:

def get_hyponyms(w):
    sys.stdout.write("Getting hyponyms ... \r")
    hyponyms = []
    for (w1, freq) in scene_word_counter:
        if is_hypernym(w1, w):
            if w not in hypernyms:
                hyponyms = [w1]
            else:
                hyponyms.append(w1)
    return hyponyms

import sys
import json
from collections import Counter

question_word_counter = Counter()
scene_word_counter = Counter()

print("Going through data ...")
for idx, entry in enumerate(data):
    program = entry[2]
    for sub_program in program:
        _, func, arguments = parse_program(sub_program)
        if func == 'relate_inv_name' or func == 'relate_name':
            question_word_counter.update([arguments[-1]])
        elif func == 'select':
            question_word_counter.update([arguments[-1]])
        elif func == 'verify_rel' or func == 'verify_rel_inv':
            question_word_counter.update([arguments[-1]])
        elif func == 'choose_rel' or func == 'choose_rel_inv':
            question_word_counter.update([arguments[2]])
        
        if idx % 3000000 == 0:
            sys.stdout.write("Finished {}/{} \r".format(idx, len(data)))
            
print("Going through scene graph ...")
i = 0
for k, v in scene_graph.items():
    for k_, v_ in v['objects'].items():
        scene_word_counter.update([v_['name']])
    
    i += 1
    if i % 10000 == 0:
        sys.stdout.write("Finished {}/{} \r".format(i, len(scene_graph.items())))

question_word_counter = question_word_counter.most_common()
scene_word_counter = scene_word_counter.most_common()

from multiprocessing import Pool
import multiprocessing

cores = multiprocessing.cpu_count()
pool = Pool(cores)

concepts = [w for (w, freq) in question_word_counter]
r = pool.map(get_hyponyms, concepts)

print("Loading scene vocabulary ...")
with open('meta_info/scene_vocab.json', 'w') as f:
    scene_vocabs = [w for (w, _) in scene_word_counter]
    json.dump(scene_vocabs, f)

print("Loading question vocabulary ...")
with open('meta_info/question_vocab.json', 'w') as f:
    question_vocabs = [w for (w, _) in question_word_counter]
    json.dump(question_vocabs, f)  

Going through data ...
Finished 0/14305328
Finished 0/14305328
Finished 0/14305328
Finished 0/14305328
Finished 3000000/14305328
Finished 3000000/14305328
Finished 6000000/14305328
Finished 9000000/14305328
Finished 9000000/14305328
Finished 9000000/14305328
Finished 12000000/14305328
Finished 12000000/14305328
Going through scene graph ...
Finished 10000/74942
Finished 20000/74942
Finished 30000/74942
Finished 40000/74942
Finished 50000/74942
Finished 60000/74942
Finished 70000/74942
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Gettin

Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hypon

Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hypon

Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hyponyms ...
Getting hypon

In [14]:
import re
import sys

hypernym = {}

for split in ['train', 'val']:
    print("Loading {} scene graphs ...".format(split))
    with open('/../../nobackup/s144454/dataset/scene_graphs/{}_sceneGraphs.json'.format(split)) as f:
        scene_graph = json.load(f)
    
    print("Loading {} programs ...".format(split))
    with open('/../../nobackup/s144454/processed/{}_balanced_programs.json'.format(split)) as f:
        data = json.load(f)
        
        print("Going through data")
        i = 0
        for entry in data:
            imageId = entry[0]
            graph = scene_graph[imageId]
            program = entry[2]
            for sub_program in program:
                result, func, arguments = parse_program(sub_program)
                if func == 'relate_inv_name' or func == 'relate_name':
                    name = arguments[-1]
                elif func == 'select':
                    name = arguments[-1]
                elif func == 'verify_rel' or func == 'verify_rel_inv':
                    name = arguments[-1]
                elif func == 'choose_rel' or func == 'choose_rel_inv':
                    name = arguments[1]
                else:
                    continue
                if re.match(r'\[[0-9]+\]', result):
                    for r in json.loads(result):
                        eq = graph['objects'][str(r)]['name']
                        if name not in hypernym:
                            hypernym[name] = set([eq])
                        else:
                            hypernym[name].add(eq)
                            
            i += 1
            if i % 10000 == 0:
                sys.stdout.write("Finished {}/{} entries \r".format(i, len(data)))
    
    print("Finished {}/{} entries".format(i, len(data)))
            
        
print("Cleaning hyponyms")
#keys = hypernym.keys()
for k in list(hypernym.keys()):
    tmp = list(hypernym[k])

    if k in tmp:
        tmp.remove(k)

    if len(tmp) == 0:
        del hypernym[k]
    else:
        hypernym[k] = tmp

print("Dumping hyponyms to file")
with open('/../../nobackup/s144454/processed/GQA_hypernym.json', 'w') as f:
    json.dump(hypernym, f, indent=2)
    
print("Done!")

Loading train scene graphs ...
Loading train programs ...
Going through data
Loading val scene graphs ...es 
Loading val programs ...
Going through data
Cleaning hyponyms32060 entries 
Dumping hyponyms to file!


In [19]:
import json

objects = {}
for split in ['train', 'val']:
    print("Loading {} scene graphs ...".format(split))
    with open('/../../nobackup/s144454/dataset/scene_graphs/{}_sceneGraphs.json'.format(split)) as f:
        scene_graph = json.load(f)
    
    i = 0
    for s_k, s_v in scene_graph.items():
        objects[s_k] = {}

        attributes = []
        if 'weather' in s_v:
            attributes.append(s_v['weather'])
        if 'location' in s_v:
            attributes.append(s_v['location'])

        objects[s_k]["0"] = {'x': 0, 'y': 0, 'w': s_v['width'], 'h': s_v['height'],
                            'relations': [], 'name': 'n/a', 'attributes': attributes}   
        for o_k, o_v in s_v['objects'].items():
            objects[s_k][o_k] = o_v
            if o_v['name'] in ['bedroom', 'kitchen', 'bathroom', 'living room']:
                objects[s_k]["0"]['attributes'].append(o_v['name'])
            elif o_v['name'] in ['road', 'sidewalk', 'field', 'beach', 'park', 'grass', 'farm', 'ocean', 'lake', 'street', 'train station', 'hotel room', 'church']:
                objects[s_k]["0"]['attributes'].append(o_v['name'])
                
        i += 1
        if i % 10000 == 0:
            sys.stdout.write("Finished {}/{} entries \r".format(i, len(scene_graph.items())))

    print("Finished {}/{} entries".format(i, len(scene_graph.items())))
    print("Dumping bounding boxes to file")
    with open('/../../nobackup/s144454/processed/{}_bounding_box.json'.format(split), 'w') as f:
        json.dump(objects, f, indent=2)
#print("Dumping bounding boxes to file")
#with open('/../../nobackup/s144454/processed/trainval_bounding_box.json', 'w') as f:
#    json.dump(objects, f, indent=2)

print("Done!")

Loading train scene graphs ...
Dumping bounding boxes to file
Loading val scene graphs ...
Dumping bounding boxes to file
Done!


In [ ]:
with open('/../../nobackup/s144454/processed/val_bounding_box.json') as f:
    val_bbox = json.load(f)

In [21]:
with open('/../../nobackup/s144454/processed/testdev_balanced_programs.json') as f:
    testdev_programs = json.load(f)

testdev_id = set()
for prog in testdev_programs:
    testdev_id.add(prog[0])

with open('/../../nobackup/s144454/processed/testdev_id.json', 'w') as f:
    json.dump(list(testdev_id), f)

In [22]:
#with open('spatial/gqa_spatial_info.json') as f:
#    image_data = json.load(f)
    
with open('/../../nobackup/s144454/dataset/images/objects/gqa_objects_info.json') as f:
    image_data = json.load(f)

In [49]:
data = []
for im_id, v in image_data.items():
    data.append({'width': v['width'], 'height': v['height'], 'image_id': im_id, 'coco_id': None, 'flickr_id': None, 'url': '/../../nobackup/s144454/dataset/images/images/{}.jpg'.format(im_id)})
    
with open('/../../nobackup/s144454/processed/image_data.json', 'w') as f:
    json.dump(data, f)


In [43]:
#with open('sceneGraphs/attributes.json') as f:
#    attributes = set(json.load(f))
#
#with open('sceneGraphs/objects.json') as f:
#    objects = set(json.load(f))
#    
#with open('sceneGraphs/predicates.json') as f:
#    predicates = set(json.load(f))
import sys
import json

attributes, objects, predicates = (set() for i in range(3))

for split in ['train', 'val']:
    print("Loading {} scene graphs ...".format(split))
    with open('/../../nobackup/s144454/dataset/scene_graphs/{}_sceneGraphs.json'.format(split)) as f:
        scene_graph = json.load(f)
    
    print("Going through {} scene graph data".format(split))
    for i, entry in enumerate(scene_graph.values()):
        for obj in entry['objects'].values():
            objects.add(obj['name'])
            for attribute in obj['attributes']:
                attributes.add(attribute)
            for relation in obj['relations']:
                predicates.add(relation['name'])
        
        i += 1
        if i % 10000 == 0:
            sys.stdout.write("Finished {}/{} entries \r".format(i, len(scene_graph.items())))

    print("Finished {}/{} entries".format(i, len(scene_graph.items())))

print("Done!")

Loading train scene graphs ...
Going through train scene graph data
Finished 74942/74942 entries 
Loading val scene graphs ...
Going through val scene graph data
Finished 10696/10696 entries 
Done!


In [44]:
obj_data = []
attr_data = []
relation_data = []
scene_graph_data = []
attr_id = 0
relation_id = 0
i = 0

for split in ['train', 'val']:
    print("Loading {} bounding boxes ...".format(split))
    with open('/../../nobackup/s144454/processed/{}_bounding_box.json'.format(split)) as f:
        bboxes = json.load(f)
        
        print("Going through {} bounding box data".format(split))
        for k, v in bboxes.items():
            obj_data.append({'image_id': k, 'objects': []})
            attr_data.append({'image_id': k, 'attributes': []})
            scene_graph_data.append({'image_id': k, 'objects': [], 'attributes': [], 'relationships': []})
            relation_data.append({'image_id': k, 'relationships': []})
            
            for bk, bv in v.items():
                if bv['name'] in objects:
                    obj_data[-1]['objects'].append({'synsets': [], 'h': bv['h'], 'w': bv['w'], 
                                                    'x': bv['x'], 'y': bv['y'], 'object_id': bk, 
                                                    'names': [bv['name']]})
                    
                    tmp_attributes = list(set(bv['attributes']) & attributes)
                    if tmp_attributes:
                        attr_data[-1]['attributes'].append({'synsets': [], 'h': bv['h'], 'w': bv['w'], 
                                                             'x': bv['x'], 'y': bv['y'], 'object_id': bk, 
                                                             'attributes': tmp_attributes})
                        scene_graph_data[-1]['attributes'].append({'image_id': k, 'attribute_id': attr_id, 
                                                                   'attribute': {'synsets': [], 'h': bv['h'], 'w': bv['w'], 
                                                                                 'x': bv['x'], 'y': bv['y'], 'object_id': bk, 
                                                                                 'attributes': tmp_attributes}})
                        attr_id += 1

                    
            scene_graph_data[-1]['objects'] = obj_data[-1]['objects']
            
            
            for bk, bv in v.items():
                if bv['name'] in objects:
                    for rels in bv['relations']:
                        if v[rels['object']]['name'] in objects and rels['name'] in predicates and rels['name'] not in ['to the left of', 'to the right of']:
                            bs = v[rels['object']]
                            relation_data[-1]['relationships'].append({'predicate': rels['name'], 
                                                                      'object': {'synsets': [], 'h': bv['h'], 'w': bv['w'], 
                                                                        'x': bv['x'], 'y': bv['y'], 'object_id': bk, 
                                                                        'names': [bv['name']]},
                                                                      'subject': {'synsets': [], 'h': bs['h'], 'w': bs['w'], 
                                                                        'x': bs['x'], 'y': bs['y'], 'object_id': rels['object'], 
                                                                        'names': [bs['name']]}
                                                                     })
                            scene_graph_data[-1]['relationships'].append({'object_id': bk, 'subject_id': rels['object'], 'predicate': rels['name'], 'synset': [], 'relationship_id': relation_id})
                            relation_id += 1
                            
            i += 1
            if i % 10000 == 0:
                sys.stdout.write("Finished {}/{} entries \r".format(i, len(bboxes.items())))

    print("Finished {}/{} entries".format(i, len(bboxes.items())))
          
print("Dumping data into files ...")
with open('/../../nobackup/s144454/processed/data/objects.json', 'w') as f:
    json.dump(obj_data, f)
with open('/../../nobackup/s144454/processed/data/attributes.json', 'w') as f:
    json.dump(attr_data, f)
with open('/../../nobackup/s144454/processed/data/relationships.json', 'w') as f:
    json.dump(relation_data, f)
with open('/../../nobackup/s144454/processed/data/scene_graphs.json', 'w') as f:
    json.dump(scene_graph_data, f)
    
print("Done!")

Loading train bounding boxes ...
Going through train bounding box data
Finished 74942/74942 entries 
Loading val bounding boxes ...
Going through val bounding box data
Finished 160580/85638 entries 
Dumping data into files ...
Done!


In [ ]:
relation_data = []

for split in ['train', 'val']:
    with open('sceneGraphs/{}_bounding_box.json'.format(split)) as f:
        bboxes = json.load(f)
        for k, v in bboxes.items():
            relation_data.append({'image_id': k, 'relationships': []})
            for bk, bv in v.items():
                if bv['name'] in objects:
                    for rels in bv['relations']:
                        if v[rels['object']]['name'] in objects and rels['name'] in predicates and rels['name'] not in ['to the left of', 'to the right of']:
                            bs = v[rels['object']]
                            relation_data[-1]['relationships'].append({'predicate': rels['name'], 
                                                                      'object': {'synsets': [], 'h': bv['h'], 'w': bv['w'], 
                                                                        'x': bv['x'], 'y': bv['y'], 'object_id': bk, 
                                                                        'names': [bv['name']]},
                                                                      'subject': {'synsets': [], 'h': bs['h'], 'w': bs['w'], 
                                                                        'x': bs['x'], 'y': bs['y'], 'object_id': rels['object'], 
                                                                        'names': [bs['name']]}
                                                                     })

In [ ]:
with open('/datadrive/wenhu/bottom-up-attention/data/gqa/relationships.json', 'w') as f:
    json.dump(relation_data, f)

In [45]:
class Node(object):
    def __init__(self):
        self.name = "?"
        self.attribute = []
        self.no_attribute = []
        self.neighbors = []
    
    def add_attr(self, name):
        self.attribute.append(name)
    
    def add_name(self, name):
        self.name = name
    
    def add_no_attr(self, name):
        self.no_attribute.append(name)
    
    def add_neighbor(self, rel, node):
        self.neighbors.append((rel, node))
    
    def __str__(self):
        return "{}@(attribute:{},no_attribute:{})".format(self.name, self.attribute, self.no_attribute)

In [69]:
def get_index(string):
    index = re.search(r'\[.+\]', string).group()[1]
    return int(index)

In [73]:
print("Loading train programs ...")
with open('/../../nobackup/s144454/processed/train_balanced_programs.json') as f:
    data = json.load(f)

buffers = []
for d in data:
    print(d[2])
    for sub_prog in d[2]:
        _, func, args = parse_program(sub_prog)
        if func == "select":
            node = Node()
            node.add_name(args[0])
            buffers.append(node)
        elif func == "filter":
            tmp = buffers[get_index(args[0])]
            tmp.add_attr(args[1])
            buffers.append(None)
        elif func == 'filter_not':
            tmp = buffers[get_index(args[0])]
            tmp.add_no_attr(args[1])
            buffers.append(None)
        elif func == 'filter_h':
            tmp = buffers[get_index(args[0])]
            tmp.add_attr(args[1])
            buffers.append(None)
        elif func == 'filter_v':
            tmp = buffers[get_index(args[0])]
            tmp.add_attr(args[1])
            buffers.append(None)
        elif func == 'relate':
            node = Node()
            buffers[get_index(args[0])].add_neighbor(args[1], node)
            buffers.append(node)
        elif func == 'relate_name':
            node = Node()
            node.add_name(attr[2])
            buffers[get_index(args[0])].add_neighbor(args[1], node)
            buffers.append(node)
        elif func == 'relate_inv':
            node = Node()
            node.add_neighbor(args[1], buffers[get_index(args[0])])
            buffers.append(node)
        elif func == 'relate_inv_name':
            node = Node()
            node.add_name(args[2])
            node.add_neighbor(args[1], buffers[get_index(args[0])])
            buffers.append(node)
        elif func == 'verify':
            node = buffers[get_index(args[0])]
            node.add_attr(args[1])
            print(node)
        elif func == 'exist':
            print(buffers[get_index(args[0])])
    break

Loading train programs ...
['[2486325]=select(sky)', '?=filter([0], dark)', '?=exist([1])']
None


In [75]:
import json

with open('/../../nobackup/s144454/processed/val_bounding_box.json') as f:
    gt_data = json.load(f)

with open('/../../nobackup/s144454/processed/train_bounding_box.json') as f:
    gt_data.update(json.load(f))

In [ ]:
with open('extracted_features/val_mapping_pred.json') as f:
    my_pred_data = json.load(f)

with open('sceneGraphs/bbox_feature_mapping.json') as f:
    provided_pred_data = json.load(f)

In [ ]:
import Constants

recall_hit, recall_fail = 0, 0
precision_hit, precision_fail = 0, 0
pred_nums = []
true_nums = []

for k, _ in my_pred_data.items():
    #pred_bbox = provided_pred_data[k]
    pred_bbox = my_pred_data[k]
    gt_bbox = gt_data[k]
    pred_nums.append(len(pred_bbox))
    true_nums.append(len(gt_bbox))
    for _, gt_v in gt_bbox.items():
        success = False
        for _, p in pred_bbox.items():
            if Constants.intersect((p['x'], p['y'], p['w'], p['h']), (gt_v['x'], gt_v['y'], gt_v['w'], gt_v['h'])) > 0.4:
                success = True
                break
        if success:
            recall_hit += 1
        else:
            recall_fail += 1
    
    for _, p in pred_bbox.items():
        success = False
        for gt_k, gt_v in gt_bbox.items():
            if Constants.intersect((p['x'], p['y'], p['w'], p['h']), (gt_v['x'], gt_v['y'], gt_v['w'], gt_v['h'])) > 0.4:
                success = True
                break
        if success:
            precision_hit += 1
        else:
            precision_fail += 1

recall = recall_hit / (recall_hit + recall_fail + 0.)
precision = precision_hit / (precision_hit + precision_fail + 0.)
f1 = 2 * recall * precision / (recall + precision)
print "final recall = {} precision = {} f1 = {}".format(recall, precision, f1)
print "average proposed bounding boxes = {} vs true boxes = {}".format(sum(pred_nums) / len(pred_nums), sum(true_nums) / len(true_nums))


In [ ]:
from torch import nn
import torch

class GCN(nn.Module):
    def __init__(self, hidden_dim):
        super(GCN, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer = 5
        self.propagate = []
        for _ in range(self.layer):
            self.propagate.append(nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                                nn.Sigmoid(),
                                                nn.Linear(hidden_dim, hidden_dim),
                                                nn.Sigmoid()))

        self.projection = nn.Sequential(nn.Linear(hidden_dim, 2 * hidden_dim),
                                      nn.Sigmoid(),
                                      nn.Dropout(0.2),
                                      nn.Linear(2 * hidden_dim, hidden_dim),
                                      nn.Sigmoid())
        
    def forward(self, input_seq, masks):
        for i, mask in enumerate(masks):
            # input_seq -> batch x length x dim
            batch_size = input_seq.size(0)
            feed = self.propagate[i](input_seq.view(-1, self.hidden_dim)).view(batch_size, -1, self.hidden_dim)
            # feed -> batch x length x dim, mask -> batch x length x length
            input_seq = torch.sigmoid(torch.bmm(mask, feed))
        
        return self.projection(input_seq)

In [ ]:
model = GCN(128)

batch_size = 8
length = 5
input_seq = torch.FloatTensor(batch_size, length, 128).fill_(1)
mask1 = torch.eye(length)
mask1[1, 0] = 1
mask1[3, 2] = 1
mask1 = mask1[None, :, :].repeat(batch_size, 1, 1)

mask2 = torch.eye(length)
mask2[4, 1] = 1
mask2[4, 2] = 1
mask2 = mask2[None, :, :].repeat(batch_size, 1, 1)

mask = [mask1, mask2]

output = model(input_seq, mask)

In [ ]:
new_data = []
for sub in submission_id:
    if sub not in data_map:
        new_data.append({'questionId': sub, 'prediction': 'yes'})
    else:
        entry = data_map[sub]
        if entry['prediction'] is None:
            new_data.append({'questionId': sub, 'prediction': 'none'})
        else:
            new_data.append(entry)
    
with open('sceneGraphs/submission_results_wo_none.json', 'w') as f:
    json.dump(new_data, f)

In [ ]:
with open('questions/submission_programs_pred.json') as f:
    data = json.load(f)

In [ ]:
for i, program in enumerate(data):
    data[i][2] = filter(lambda x: len(x) > 0 and x[-1] == ")", program[2])
    for prog in program[2]:
        try:
            _, func, args = parse_program(prog)
        except Exception:
            print program[2]
            
with open('questions/submission_programs.json', 'w') as f:
    json.dump(data, f)

In [34]:
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

emb = loadGloveModel('glove/glove.6B.300d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [41]:
import json

def save(inputs, outputs):
    with open(inputs) as f:
        vocab = json.load(f)

    found, miss = 0, 0
    en_emb = np.zeros((len(vocab), 300), 'float32')
    for w, i in vocab.items():
        if w.lower() in emb:
            en_emb[i] = emb[w.lower()]
            found += 1
        elif ' ' in w:
            for w_elem in w.split(' '):
                if w_elem.lower() in emb:
                    en_emb[i] += emb[w_elem.lower()]
        else:
            print(w)
            miss += 1

    print("found = {}, miss = {}".format(found, miss))
    np.save(outputs, en_emb)

save('nl2prog/en_vocab.json', 'models/en_emb.npy')
save('nl2prog/concept_vocab.json', 'models/concept_emb.npy')

[PAD]
[EOS]
[UNK]
tshirt
sitting-down
dry-erase
drainers
ottomen
turned-off
mooses
logoes
pianoes
hippoes
lying-down
celeries
avocadoes
mousess
burritoes
turned-on
puppys
tacoes
drainer
snowpants
asparaguss
bronwy
legoes
not-healthy
elmoes
octopodes
meatballss
cactuss
plier
pokemons
pikachus
found = 2935, miss = 34
plier
found = 1944, miss = 1


In [ ]:
import json

with open('questions/testdev_all_questions.json') as f:
    all_data = json.load(f)
    
with open('questions/testdev_balanced_questions.json') as f:
    balanced_data = json.load(f)

In [ ]:
import torch

#with open('extracted_features/train_features.bin') as f:
feats = []
feats.append(torch.load('extracted_features/train_features.bin'))
feats.append(torch.load('extracted_features/val_features.bin'))
feats.append(torch.load('extracted_features/test_features.bin'))
feats.append(torch.load('extracted_features/testdev_features_pred.bin'))
feats.append(torch.load('extracted_features/submission_features_pred.bin'))

mappings = []
with open('extracted_features/train_mapping.json') as f:
    mappings.append(json.load(f))
    
with open('extracted_features/val_mapping.json') as f:
    mappings.append(json.load(f))

with open('extracted_features/test_mapping.json') as f:
    mappings.append(json.load(f))
    
with open('extracted_features/testdev_mapping_pred.json') as f:
    mappings.append(json.load(f))
    
with open('extracted_features/submission_mapping_pred.json') as f:
    mappings.append(json.load(f))

In [ ]:
import h5py
import numpy as np
import sys

infos = {}
with h5py.File('extracted_features/gqa_objects.h5', 'w') as f:
    length = sum([len(_) for _ in mappings])
    f.create_dataset("features", (length, 48, 2048), dtype='float32')
    f.create_dataset("bboxes", (length, 48, 4), dtype='float32')

    counter = 0
    for mapping, feat in zip(mappings, feats):
        for k, v in mapping.items():
            infos[k] = {'width': v['0']['w'], 'height': v['0']['h'], 'objectsNum': len(v) - 1, 'index': counter}
            tmp_feat = np.zeros((48, 2048), 'float32')
            tmp_bbox = np.zeros((48, 4), 'float32')
            tmp_counter = 0
            for bk, bv in v.items():
                if bk == "0":
                    continue
                else:
                    tmp_feat[tmp_counter] = feat[bv['feat_idx']]
                    tmp_bbox[tmp_counter] = np.array([bv['x'], bv['y'], bv['w'], bv['h']], 'float32')
                    tmp_counter += 1

                if tmp_counter >= 48:
                    break

            f['features'][counter] = tmp_feat
            f['bboxes'][counter] = tmp_bbox
            counter += 1
            sys.stdout.write('finished {}/{} \r'.format(counter, length))

with open('extracted_features/gqa_objects_merged_info.json', 'w') as f:
    json.dump(infos, f, indent=2)

In [ ]:
database = set(infos.keys())

In [ ]:
with open('Attention/trainval_balanced_inputs.json') as f:
    d = json.load(f)

In [1]:
with open('questions/submission_all_questions.json') as f:
    keys = json.load(f).keys()

NameError: name 'json' is not defined

In [8]:
with open('questions/val_all_questions.json') as f:
    val_keys = json.load(f).keys()

In [9]:
print len(val_keys)

2011853


In [11]:
with open('../gqa-questions/test_all_questions.json') as f:
    test_keys = json.load(f).keys()

In [10]:
len(set(submission_all_keys) & set(val_keys))

2011853

In [12]:
len(set(submission_all_keys) & set(test_keys))

1340048

In [13]:
len(set(submission_all_keys) - (set(val_keys) | set(test_keys)))

885623

In [14]:
with open('../gqa-questions/testdev_all_questions.json') as f:
    testdev_keys = json.load(f).keys()

In [18]:
nedded_submission_keys = set(submission_all_keys) - (set(val_keys) | set(test_keys) | set(testdev_keys))

In [20]:
with open('../gqa-questions/train_all_questions/train_all_questions_0.json') as f:
    train_keys = json.load(f).keys()

In [19]:
results = {}
for k in nedded_submission_keys:
    results[k] = keys[k]
    
with open('questions/needed_all_questions.json', 'w') as f:
    json.dump(results, f)

In [2]:
import json

with open('questions/submission_all_questions.json') as f:
    submission_keys = json.load(f).keys()

In [8]:
with open('questions/submission_all_questions.json') as f:
    submissions = json.load(f)

In [3]:
with open('questions/trainval_all_programs.json') as f:
    trainval_programs = json.load(f)

with open('questions/testdev_all_programs.json') as f:
    trainval_programs += json.load(f)

In [5]:
submission_keys = set(submission_keys)
contained_keys = []
already_existed_programs = []
for program in trainval_programs:
    if program[-2] in submission_keys:
        already_existed_programs.append(program)
        contained_keys.append(program[-2])

In [9]:
with open('questions/overlapped_submission_programs.json', 'w') as f:
    json.dump(already_existed_programs, f, indent=2)

In [10]:
contained_keys = set(contained_keys)
needed_submission = {}
for k, v in submissions.items():
    if k not in contained_keys:
        needed_submission[k] = v

In [12]:
with open('questions/needed_submission_questions.json', 'w') as f:
    json.dump(needed_submission, f)

In [13]:
import json

with open('sceneGraphs/train_bounding_box.json') as f:
    scene_graph = json.load(f)

with open('sceneGraphs/val_bounding_box.json') as f:
    scene_graph.update(json.load(f))

In [28]:
from collections import Counter

word_counter = Counter()
mapping = {}

for k, v in scene_graph.items():
    mapping[k] = set()
    for bk, bv in v.items():
        if bv['name'] != 'n/a':
            word_counter.update([bv['name']])
            mapping[k].add(bv['name'])
            mapping[k] = mapping[k] | set(bv['attributes'])
            word_counter.update(bv['attributes'])
            for rel in bv['relations']:
                mapping[k] = mapping[k] | set([rel['name']])
                word_counter.update([rel['name']])
    mapping[k] = list(mapping[k])

In [29]:
with open('sceneGraphs/concepts.json', 'w') as f:
    json.dump(mapping, f, indent=2)

In [1]:
from GQA import GQA_v2

In [ ]:
basic_kwargs = dict(length=args.length, object_info=os.path.join('sceneGraphs', args.object_info), num_regions=args.num_regions, \
                    vocab=vocab, answer=answer, max_layer=5, num_tokens=30, \
                    spatial_feat='sceneGraphs/gqa_spatial.h5', spatial_info='sceneGraphs/gqa_spatial_merged_info.json')

train_dataset = GQA_v2(split=train_split, mode='train', soft_label=args.soft_label, threshold=args.threshold, folder=args.data, **basic_kwargs)

In [14]:
import json
with open('questions/trainval_all_programs.json') as f:
    data = json.load(f)
with open('sceneGraphs/trainval_bounding_box.json') as f:
    bounding_box = json.load(f)

In [3]:
def parse_program(string):
    if '=' in string:
        result, function = string.split('=')
    else:
        function = string
        result = "?"

    func, arguments = function.split('(')
    if len(arguments) == 1:
        return result, func, []
    else:
        arguments = list(map(lambda x: x.strip(), arguments[:-1].split(',')))
        return result, func, arguments

In [25]:
import copy
import sys

new_program = []
succ, fail = 0, 0
for program in data:
    if 'verify_rel_inv' in program[2][-1]:
        last_program = program[2][-1]
        
        r, func, args = parse_program(last_program)
        dependency, relation, name = args
        
        target = int(dependency[1:-1])
        target = program[2][target].split('=')[0][1:-1]
        
        question = program[1]
        scene = bounding_box[program[0]]
        new_relation = None
        new_name = None
        new_id = None
        for k, obj in scene.items():
            for rel in obj['relations']:
                if rel['object'] == target:
                    new_relation = rel['name']
                    new_name = obj['name']
                    new_id = k
                    break
        #assert new_relation is not None, program
        #assert new_name is not None
        #assert new_id is not None
        if new_relation:
            replicate = copy.deepcopy(program)
            replicate[1] = replicate[1].replace(relation, new_relation)
            replicate[1] = replicate[1].replace(name, new_name)
            replicate[2][-1] = '[{}]=verify_rel_inv({},{},{})'.format(new_id, dependency, new_relation, new_name)
            replicate[-1] = 'yes'
            replicate[-2] = 'x' + replicate[-2]
            new_program.append(replicate)
            succ += 1
        else:
            fail += 1
            pass
        sys.stdout.write('success={}, fail={} \r'.format(succ, fail))
        #print relation in question

In [27]:
new_program[0]

[u'2346563',
 u'Is the man to the left of the boat?',
 [u'[898277]=select(boat)', '[898273]=verify_rel_inv([0],to the left of,man)'],
 u'x0890035',
 'yes']

In [26]:
with open('questions/trainval_calibrated_programs.json', 'w') as f:
    json.dump(new_program, f, indent=2)

In [1]:
import json

with open('nl2prog/v2_mscoco_val2014_annotations.json') as f:
    annotation = json.load(f)

with open('nl2prog/v2_OpenEnded_mscoco_val2014_questions.json') as f:
    question = json.load(f)

In [6]:
question_mapping = {q['question_id']: (q['image_id'], q['question']) for q in question['questions']}
annotation_mapping = {q['question_id']: q[] for q in question['annotations']}
#for q in question['questions']:
#    print(q['question_id'], q['question'], q['image_id'])
#    break

262148000 Where is he looking? 262148
